In [1]:
print("""
@File         : ch02_finding_and_wrangling_time_series_data.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-03-08 23:25:05
@Email        : cuixuanstephen@gmail.com
@Description  : 时间序列的发现与整理
""")


@File         : ch02_finding_and_wrangling_time_series_data.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-03-08 23:25:05
@Email        : cuixuanstephen@gmail.com
@Description  : 时间序列的发现与整理



In [1]:
import pandas as pd

In [23]:
year_joined = pd.read_csv('../data/year_joined.csv')

In [24]:
# 可以看出来每个用户只有一个账户
year_joined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


In [25]:
year_joined.groupby('user').count()

,userStats,yearJoined
user,,
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
995,1,1
996,1,1
997,1,1


In [49]:
emails = pd.read_csv('../data/emails.csv', parse_dates=['week'])

In [50]:
emails[emails.emailsOpened < 1]

,emailsOpened,user,week


In [51]:
emails[emails.user == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04
25465,3.0,998.0,2017-12-11
25466,3.0,998.0,2017-12-18
25467,3.0,998.0,2018-01-01
25468,3.0,998.0,2018-01-08
25469,2.0,998.0,2018-01-15
25470,3.0,998.0,2018-01-22
25471,2.0,998.0,2018-01-29
25472,3.0,998.0,2018-02-05
25473,3.0,998.0,2018-02-12


可以看出来缺少了几周数据，从 2017 年 12 月 18 日之后，就没有该月份的任何邮件事件了。

In [61]:
(
    max(emails[emails.user == 998]["week"]) - min(emails[emails.user == 998]["week"])
).days / 7

25.0

In [63]:
emails[emails.user == 998].shape

(24, 3)

可以看出数据只有 24 条，但是应该有 26 条。

你可能会感到惊讶，考虑到我们刚刚执行的减法，我们需要 26 而不是 25，但这是一个不完整的计算。当你处理时间序列数据时，在进行这种减法之后，你应该始终问自己的一件事是是否应该加 1 。换句话说，你是否减去了你想要统计的位置？

有些确实数据是在有记录之前或者之后，我们没有办法进行填补。但是第一条记录和最后一条记录之间的数据非空数据可以补全。

In [64]:
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))

In [97]:
all_email = (
    emails.set_index(["week", "user"])
    .reindex(complete_idx, fill_value=0)
    .reset_index(names=["week", "user"])
)
# all_email.columns = ["week", "user", "emailsOpened"]

In [100]:
all_email[all_email.user == 998].sort_values('week')

,week,user,emailsOpened
66835,2015-02-09,998.0,0.0
71686,2015-02-16,998.0,0.0
88395,2015-02-23,998.0,0.0
11318,2015-03-02,998.0,0.0
29644,2015-03-09,998.0,0.0
...,...,...,...
25871,2018-04-30,998.0,3.0
28027,2018-05-07,998.0,3.0
18864,2018-05-14,998.0,3.0
46892,2018-05-21,998.0,3.0
